# Imbalanced Data
***

이번 챕터에서는 비대칭 데이터(Imbalanced Data) 해결 방법에 대해 알아보도록 하겠습니다. 비대칭 데이터는 일반적으로 분류(classification) 문제 해결에서 예측 대상 클래스들이 불규칙할 경우 문제를 일으킵니다.

예를 들어 1만명의 환자 데이터로 암 발병 여부를 예측한다고 했을 때 암에 걸리지 않은 환자가 9900명일 때를 가정해봅시다. 평가지표는 정확도(accuracy)일 때 모델을 따로 구성하지 않고 10,000명의 사람이 모두 암에 걸리지 않았ek고 예측하면 정확도는 무려 99%를 달성하게 됩니다. 즉 예측의 정확도(accuracy)는 높아지지만 재현율(recall : 실제 양성을 양성이라고 옳게 예측할 확률)이 매우 낮을 것입니다.

하지만 이런 식으로 환자의 암 발병 여부를 예측할 때 실제 암에 걸린 사람을 암에 걸리지 않았다고 예측하면 암 환자의 상태는 더 위독해질 것입니다.

이렇듯 비대칭 데이터 문제를 해결할 때에는 데이터의 샘플링과 올바른 평가지표 사용이 중요하다고 볼 수 있습니다.
***

데이터 샘플링의 방식은 아래와 같이 크게 두 가지가 존재합니다.

1. Over Sampling = 데이터의 수가 소수인 클래스의 데이터를 추가적으로 더 만들어낸다.


2. Under Sampling = 데이터의 수가 다수인 클래스의 데이터를 일부만 사용한다.

일반적으로 Over Sampling이 성능이 더 좋은 경우가 많아 자주 사용됩니다.
***
비대칭 데이터 문제 해결시 올바른 평가지표로는 아래와 같은 기준들이 권장되고 있습니다.

1. Confusion Matrix

2. Precision

3. Recall

4. F1 Score

5. ROC AUC

***
각종 방법들을 사용할 때는 학습 데이터에만 샘플링을 적용하여야 합니다. 실제 우리가 풀어야할 테스트 데이터에는 샘플링을 적용하면 올바른 테스트가 진행되지 못하기 때문입니다.

***

먼저 Over Sampling 기법들에 대해 알아보도록 하겠습니다. 

대표적인 Over Sampling 기법인 SMOTE와 RandomOverSampler을 사용해 비교해보도록 하죠.


데이터는 kaggle의 신용카드 사기 검출 데이터를 사용하겠습니다.

In [1]:
import pandas as pd
from imblearn.over_sampling import *
from imblearn.under_sampling import *

from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

In [2]:
df = pd.read_csv("C:/Users/KI/Vacation/2019summer/python_machinelearning_guide/creditcard.csv")

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [22]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

두 클래스의 개수 차이가 압도적으로 나는 것을 알 수 있습니다.

In [4]:
X = df.iloc[:, 1:-1]
y = df['Class']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1219)

Sampling의 효과만 알아보기 위해 별도의 변수 전처리는 진행하지 않도록 하겠습니다.

우선 baseline 모델 구성을 위해서 LightGBM으로 성능을 확인해보겠습니다.

In [7]:
lgbm = LGBMClassifier(random_state = 1219)

In [8]:
lgbm.fit(X_train, y_train)

LGBMClassifier(random_state=1219)

In [18]:
baseline_class = lgbm.predict(X_test)
baseline_prob = lgbm.predict_proba(X_test)[:, 1]

In [19]:
base_acc = round(accuracy_score(y_test, baseline_class), 4)
base_precision = round(precision_score(y_test, baseline_class), 4)
base_recall = round(recall_score(y_test, baseline_class), 4)
base_f1 = round(f1_score(y_test, baseline_class), 4)
base_roc_auc = round(roc_auc_score(y_test, baseline_prob), 4)

In [20]:
print(f'baseline 모델의 정확도는 {base_acc * 100}%')
print(f'baseline 모델의 정밀도는 {base_precision * 100}%')
print(f'baseline 모델의 재현율는 {base_recall * 100}%')
print(f'baseline 모델의 f1 score는 {base_f1 * 100}%')
print(f'baseline 모델의 roc auc socre는 {base_roc_auc * 100}%')

baseline 모델의 정확도는 99.68%
baseline 모델의 정밀도는 20.810000000000002%
baseline 모델의 재현율는 42.86%
baseline 모델의 f1 score는 28.02%
baseline 모델의 roc auc socre는 54.03%


baseline 모델의 경우 정확도를 제외한 모든 지표에서 낮은 성능을 보이고 있습니다.
***

이제 SMOTE 기법을 적용해 학습 및 평가를 해보도록 하겠습니다.

In [23]:
smote = SMOTE(random_state = 1220)

In [24]:
X_smote, y_smote = smote.fit_sample(X_train, y_train)

In [27]:
print(f'원본 데이터의 shape = {X.shape}')
print(f'SMOTE 적용 후 데이터의 shape = {X_smote.shape}')

원본 데이터의 shape = (284807, 29)
SMOTE 적용 후 데이터의 shape = (454874, 29)


SMOTE 적용 후 데이터가 2배로 늘어난 것을 알 수 있습니다.

이번에는 예측값의 클래스 분포를 확인해보겠습니다.

In [30]:
print(f'SMOTE 적용 후 클래스 분포 :\n{pd.Series(y_smote).value_counts()}')

SMOTE 적용 후 클래스 분포 :
1    227437
0    227437
Name: Class, dtype: int64


SMOTE 적용 후 예측값의 클래스가 1대1로 동일해진 것을 알 수 있습니다. 이제 생성된 데이터로 학습한 후 성능을 평가해보겠습니다.

In [31]:
lgbm = LGBMClassifier(random_state = 1219)

In [32]:
lgbm.fit(X_smote, y_smote)

LGBMClassifier(random_state=1219)

In [33]:
smote_class = lgbm.predict(X_test)
smote_prob = lgbm.predict_proba(X_test)[:, 1]

In [36]:
SMOTE_acc = round(accuracy_score(y_test, smote_class), 4)
SMOTE_precision = round(precision_score(y_test, smote_class), 4)
SMOTE_recall = round(recall_score(y_test, smote_class), 4)
SMOTE_f1 = round(f1_score(y_test, smote_class), 4)
SMOTE_roc_auc = round(roc_auc_score(y_test, smote_prob), 4)

In [37]:
print(f'SMOTE 모델의 정확도는 {SMOTE_acc * 100}%')
print(f'SMOTE 모델의 정밀도는 {SMOTE_precision * 100}%')
print(f'SMOTE 모델의 재현율는 {SMOTE_recall * 100}%')
print(f'SMOTE 모델의 f1 score는 {SMOTE_f1 * 100}%')
print(f'SMOTE 모델의 roc auc socre는 {SMOTE_roc_auc * 100}%')

SMOTE 모델의 정확도는 99.86%
SMOTE 모델의 정밀도는 52.21%
SMOTE 모델의 재현율는 84.52%
SMOTE 모델의 f1 score는 64.55%
SMOTE 모델의 roc auc socre는 93.08999999999999%


SMOTE 적용 후 모든 평가 지표에서 성능이 개선된 것을 알 수 있습니다. 

이번에는 RandomOverSampler를 사용해보겠습니다.

In [38]:
ros = RandomOverSampler(random_state = 1220)

In [39]:
X_ros, y_ros = ros.fit_sample(X_train, y_train)

In [45]:
print(f'RandomOverSampler 적용 후 클래스 분포 :\n{pd.Series(y_ros).value_counts()}')

RandomOverSampler 적용 후 클래스 분포 :
1    227437
0    227437
Name: Class, dtype: int64


SMOTE와 동일하게 데이터가 2배로 늘어난 것을 확인하였습니다.

In [41]:
lgbm.fit(X_ros, y_ros)

LGBMClassifier(random_state=1219)

In [42]:
ros_class = lgbm.predict(X_test)
ros_prob = lgbm.predict_proba(X_test)[:, 1]

In [43]:
ros_acc = round(accuracy_score(y_test, ros_class), 4)
ros_precision = round(precision_score(y_test, ros_class), 4)
ros_recall = round(recall_score(y_test, ros_class), 4)
ros_f1 = round(f1_score(y_test, ros_class), 4)
ros_roc_auc = round(roc_auc_score(y_test, ros_prob), 4)

In [44]:
print(f'RandomOverSampler모델의 정확도는 {ros_acc * 100}%')
print(f'RandomOverSampler모델의 정밀도는 {ros_precision * 100}%')
print(f'RandomOverSampler모델의 재현율는 {ros_recall * 100}%')
print(f'RandomOverSampler모델의 f1 score는 {ros_f1 * 100}%')
print(f'RandomOverSampler모델의 roc auc socre는 {ros_roc_auc * 100}%')

RandomOverSampler모델의 정확도는 99.95%
RandomOverSampler모델의 정밀도는 81.17999999999999%
RandomOverSampler모델의 재현율는 82.14%
RandomOverSampler모델의 f1 score는 81.66%
RandomOverSampler모델의 roc auc socre는 95.89999999999999%


SMOTE 보다는 재현율이 2%가량 낮지만 나머지 지표에서는 모두 성능이 더 좋은 것을 알 수 있습니다.

해당 문제에서는 Over Sampling 적용이 좋은 성능을 보였지만 모든 문제에서 해당되는 것이 아니기 때문에 여러 요소를 고려한 후 결정을 해야합니다.
***

이번에는 Under Sampling 기법을 활용해보도록 하겠습니다.

In [47]:
ak = AllKNN()

In [48]:
X_ak, y_ak = ak.fit_sample(X_train, y_train)

In [52]:
print(f'AllKNN 적용 후 클래스 분포 :\n{pd.Series(y_ak).value_counts()}')

AllKNN 적용 후 클래스 분포 :
0    227268
1       408
Name: Class, dtype: int64


클래스가 0인 데이터의 수는 60000개 가량 줄었고 클래스가 1인 데이터도 약간 줄어든 것을 알 수 있습니다. 감소 비율은 클래스가 0인 데이터에서 더 높았지만 애초에 수가 매우 적었던 클래스 1데이터도 감소했기 때문에 어느 정도 데이터의 손실은 있다고 볼 수 있습니다.

In [41]:
lgbm.fit(X_ak, y_ak)

LGBMClassifier(random_state=1219)

In [49]:
ak_class = lgbm.predict(X_test)
ak_prob = lgbm.predict_proba(X_test)[:, 1]

In [50]:
ak_acc = round(accuracy_score(y_test, ak_class), 4)
ak_precision = round(precision_score(y_test, ak_class), 4)
ak_recall = round(recall_score(y_test, ak_class), 4)
ak_f1 = round(f1_score(y_test, ak_class), 4)
ak_roc_auc = round(roc_auc_score(y_test, ak_prob), 4)

In [51]:
print(f'AllKNN모델의 정확도는 {ak_acc * 100}%')
print(f'AllKNN모델의 정밀도는 {ak_precision * 100}%')
print(f'AllKNN모델의 재현율는 {ak_recall * 100}%')
print(f'AllKNN모델의 f1 score는 {ak_f1 * 100}%')
print(f'AllKNN모델의 roc auc socre는 {ak_roc_auc * 100}%')

AllKNN모델의 정확도는 99.95%
AllKNN모델의 정밀도는 81.17999999999999%
AllKNN모델의 재현율는 82.14%
AllKNN모델의 f1 score는 81.66%
AllKNN모델의 roc auc socre는 95.89999999999999%


이번에는 InstanceHardnessThreshold 기법을 적용해보도록 하겠습니다.

In [53]:
iht = InstanceHardnessThreshold(random_state = 1220)

In [54]:
X_iht, y_iht = iht.fit_sample(X_train, y_train)

In [60]:
print(f'InstanceHardnessThreshold 적용 후 클래스 분포 :\n{pd.Series(y_ak).value_counts()}')

InstanceHardnessThreshold 적용 후 클래스 분포 :
0    227268
1       408
Name: Class, dtype: int64


AllKNN처럼 두 클래스에서 모두 데이터가 일정 비율 감소한 것을 확인했습니다.

In [56]:
lgbm.fit(X_iht, y_iht)

LGBMClassifier(random_state=1219)

In [57]:
iht_class = lgbm.predict(X_test)
iht_prob = lgbm.predict_proba(X_test)[:, 1]

In [58]:
iht_acc = round(accuracy_score(y_test, iht_class), 4)
iht_precision = round(precision_score(y_test, iht_class), 4)
iht_recall = round(recall_score(y_test, iht_class), 4)
iht_f1 = round(f1_score(y_test, iht_class), 4)
iht_roc_auc = round(roc_auc_score(y_test, iht_prob), 4)

In [59]:
print(f'InstanceHardnessThreshold모델의 정확도는 {iht_acc * 100}%')
print(f'InstanceHardnessThreshold모델의 정밀도는 {iht_precision * 100}%')
print(f'InstanceHardnessThreshold모델의 재현율는 {iht_recall * 100}%')
print(f'InstanceHardnessThreshold모델의 f1 score는 {iht_f1 * 100}%')
print(f'InstanceHardnessThreshold모델의 roc auc socre는 {iht_roc_auc * 100}%')

InstanceHardnessThreshold모델의 정확도는 99.6%
InstanceHardnessThreshold모델의 정밀도는 21.29%
InstanceHardnessThreshold모델의 재현율는 63.1%
InstanceHardnessThreshold모델의 f1 score는 31.830000000000002%
InstanceHardnessThreshold모델의 roc auc socre는 75.92999999999999%


다른 기법들과 달리 눈에 띌만큼 성능이 개선되지 않은 것을 확인하였습니다.
***
이외에도 imblearn에는 combine, ensemble 패키지가 추가적으로 더 있습니다. 이번 챕터에서는 다루지 않았지만 관심이 있으신 분들은 공식문서를 참고해서 공부하시면 될 것 같습니다.
***
이번 챕터에서는 불균형 데이터 문제를 해결할 수 있는 여러가지 기법들에 대해서 알아보았습니다. 모든 ML 문제가 그렇듯이 정해진 해결 방법은 없습니다. 모든 방법을 시도해보고 수정해가면서 해당 문제를 가장 잘 해결할 수 있는 방법을 찾는 것이 가장 중요할 것입니다. 

다음 챕터에서는 스태킹(Stacking) 기법에 대해서 알아보도록 하겠습니다. 감사합니다.